# Description
- MCP tutorial from https://docs.llamaindex.ai/en/stable/examples/tools/mcp/

### Using Tools from an MCP Server:

In [2]:
from llama_index.tools.mcp import (
    get_tools_from_mcp_url,
    aget_tools_from_mcp_url,
)

# async
tools = await aget_tools_from_mcp_url("http://127.0.0.1:8000/sse")

In [5]:
# or:
from llama_index.tools.mcp import BasicMCPClient

client = BasicMCPClient("http://127.0.0.1:8000/sse")

tools = await aget_tools_from_mcp_url(
    "http://127.0.0.1:8000/mcp",
    client=client,
    allowed_tools=["tool1", "tool2"],
)

### Converting a Workflow to an MCP app:

In [6]:
from llama_index.core.workflow import (
    Context,
    Workflow,
    Event,
    StartEvent,
    StopEvent,
    step,
)
from llama_index.tools.mcp.utils import workflow_as_mcp


class RunEvent(StartEvent):
    msg: str


class InfoEvent(Event):
    msg: str


class LoudWorkflow(Workflow):
    """Useful for converting strings to uppercase and making them louder."""

    @step
    def step_one(self, ctx: Context, ev: RunEvent) -> StopEvent:
        ctx.write_event_to_stream(InfoEvent(msg="Hello, world!"))

        return StopEvent(result=ev.msg.upper() + "!")


workflow = LoudWorkflow()

mcp = workflow_as_mcp(workflow)

### MCP Client Usage:

#### Basic Client Operations

In [11]:
from llama_index.tools.mcp import BasicMCPClient

# Connect to an MCP server using different transports
http_client = BasicMCPClient("https://example.com/mcp")  # Streamable HTTP
sse_client = BasicMCPClient("http://127.0.0.1:8000/sse")  # Server-Sent Events
local_client = BasicMCPClient("python", args=["server.py"])  # stdio

# List available tools
tools = await sse_client.list_tools()

# Call a tool
result = await sse_client.call_tool("calculate", {"x": 5, "y": 10})

# List available resources
resources = await sse_client.list_resources()

# Read a resource
#content, mime_type = await sse_client.read_resource("config://app")

# List available prompts
prompts = await sse_client.list_prompts()

# Get a prompt
prompt_result = await sse_client.get_prompt("greet", {"name": "World"})

HTTP Request: GET http://127.0.0.1:8000/sse "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8000/messages/?session_id=65304172c7004f6692c273c5a4fa81bd "HTTP/1.1 202 Accepted"
HTTP Request: POST http://127.0.0.1:8000/messages/?session_id=65304172c7004f6692c273c5a4fa81bd "HTTP/1.1 202 Accepted"
HTTP Request: POST http://127.0.0.1:8000/messages/?session_id=65304172c7004f6692c273c5a4fa81bd "HTTP/1.1 202 Accepted"
HTTP Request: GET http://127.0.0.1:8000/sse "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8000/messages/?session_id=de9db2c4ffed487cb2f093962fbb6e97 "HTTP/1.1 202 Accepted"
HTTP Request: POST http://127.0.0.1:8000/messages/?session_id=de9db2c4ffed487cb2f093962fbb6e97 "HTTP/1.1 202 Accepted"
HTTP Request: POST http://127.0.0.1:8000/messages/?session_id=de9db2c4ffed487cb2f093962fbb6e97 "HTTP/1.1 202 Accepted"
HTTP Request: GET http://127.0.0.1:8000/sse "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:8000/messages/?session_id=50fe981affa94194bac41d871e4ffc3b "HTTP

### OAuth Authencation:

In [14]:
# MCP Client with Auth:
from llama_index.tools.mcp import BasicMCPClient

# Simple authentication with in-memory token storage
client = BasicMCPClient.with_oauth(
    "http://127.0.0.1:8000/sse",
    client_name="My App",
    redirect_uris=["http://localhost:9186/home"],
    # Function to handle the redirect URL (e.g., open a browser)
    redirect_handler=lambda url: print(f"Please visit: {url}"),
    # Function to get the authorization code from the user
    callback_handler=lambda: (input("Enter the code: "), None),
)

# Use the authenticated client
tools = await client.list_tools()

HTTP Request: GET http://127.0.0.1:8000/.well-known/oauth-authorization-server "HTTP/1.1 404 Not Found"
HTTP Request: GET http://127.0.0.1:8000/.well-known/oauth-authorization-server "HTTP/1.1 404 Not Found"
HTTP Request: POST http://127.0.0.1:8000/register "HTTP/1.1 404 Not Found"
Client registration failed
Traceback (most recent call last):
  File "c:\Users\duy\Desktop\I love Claude\RAG_Traffic_Law\TruongDuy\Lib\site-packages\mcp\client\auth.py", line 291, in _get_or_register_client
    self._client_info = await self._register_oauth_client(
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        self.server_url, self.client_metadata, self._metadata
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "c:\Users\duy\Desktop\I love Claude\RAG_Traffic_Law\TruongDuy\Lib\site-packages\mcp\client\auth.py", line 200, in _register_oauth_client
    raise httpx.HTTPStatusError(
    ...<3 lines>...
    )
httpx.HTTPStatusError: Registration failed: 404
  + E

In [ ]:
from llama_index.tools.mcp import BasicMCPClient
from mcp.client.auth import TokenStorage
from mcp.shared.auth import OAuthToken, OAuthClientInformationFull
from typing import Optional


class DefaultInMemoryTokenStorage(TokenStorage):
    """
    Simple in-memory token storage implementation for OAuth authentication.

    This is the default storage used when none is provided to with_oauth().
    Not suitable for production use across restarts as tokens are only stored
    in memory.
    """

    def __init__(self):
        self._tokens: Optional[OAuthToken] = None
        self._client_info: Optional[OAuthClientInformationFull] = None

    async def get_tokens(self) -> Optional[OAuthToken]:
        """Get the stored OAuth tokens."""
        return self._tokens

    async def set_tokens(self, tokens: OAuthToken) -> None:
        """Store OAuth tokens."""
        self._tokens = tokens

    async def get_client_info(self) -> Optional[OAuthClientInformationFull]:
        """Get the stored client information."""
        return self._client_info

    async def set_client_info(
        self, client_info: OAuthClientInformationFull
    ) -> None:
        """Store client information."""
        self._client_info = client_info


# Use custom storage
client = BasicMCPClient.with_oauth(
    "https://api.example.com/mcp",
    client_name="My App",
    redirect_uris=["http://localhost:3000/callback"],
    redirect_handler=lambda url: print(f"Please visit: {url}"),
    callback_handler=lambda: (input("Enter the code: "), None),
    token_storage=DefaultInMemoryTokenStorage(),
)